# Logistic Regression Example

In this tutorial, we'll walk through how to use our automatic differentiation engine to create and train a logistic regression model to predict diabetes. 

The data can be found [here](https://www.kaggle.com/uciml/pima-indians-diabetes-database).

Our goal here is not to create the best model to fit our data, but to simply show how to use our automatic differentiation engine to create, train, and test a linear regression model. Thus, our model will be very simplistic for pedagogical purposes.

# Data Preprocessing

We'll first take look at our data, validate it, and preprocess it to be in a form appropriate to use with our automatic differentiation engine.

Feel free to skip this section as it doesn't directly relate to how to use our automatic differentiation engine.

Let's first import some necessary libraries.

In [3]:
from autograd import Variable
import autograd
from tqdm import tqdm
import pandas as pd
import numpy as np
import random

Let's load the data. 

In [4]:
csv_path = './data/diabetes.csv'
df = pd.read_csv(csv_path)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Let's verify that there are no null values we need to worry about. 

In [5]:
assert df.isnull().sum().sum() == 0

Our model will be very simple. We'll create a logistic regression model that takes in a value from each column and will return the predicted price.

Our automatic differentiation engine expects numpy arrays. 

Let's convert the data into two numpy arrays, one for the inputs and one for the outputs.

In [6]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [7]:
input_df = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]
output_df = df['Outcome']
input_matrix = input_df.values
output_vector = output_df.values

In [8]:
type(input_matrix)

numpy.ndarray

In [9]:
input_matrix.shape

(768, 8)

In [10]:
type(output_vector)

numpy.ndarray

In [11]:
output_vector.shape

(768,)

Let's now split our data into a training set and test set. We'll use 70% of our data for training and 30% for testing. We'll manually set the random seeds for reproducibility purposes.

In [12]:
random.seed(0)
np.random.seed(0)

row_count = input_matrix.shape[0]
row_indices = list(range(row_count))
random.shuffle(row_indices)

last_training_row_position = round(row_count*0.70)
training_indices = row_indices[:last_training_row_position+1]
testing_indices = row_indices[last_training_row_position+1:]

training_input_matrix = input_matrix[training_indices]
training_output_vector = output_vector[training_indices]
testing_input_matrix = input_matrix[testing_indices]
testing_output_vector = output_vector[testing_indices]

We have 6 input columns, so we'll need to learn 6 multiplicative weights and 6 biases for our linear regression model. 

Let's initialize those weights.

In [10]:
weights = Variable(np.random.rand(6))
biases = Variable(np.random.rand(6))

First, we'll need to establish some hyperparameters for training our linear regression model. We'll be using stochastic gradient descent.

In practice, we might use various batch sizes or shuffle our training data between epochs, but we'll forgo this for the sake of simplicity as the purposes of this tutorial is to show how to use our automatic differentiation engine rather than finding the best model possible.

In [11]:
number_of_epochs = 800
learning_rate = 1e-5
sgd = autograd.optimizer.SGD(learning_rate)

Let's write our training loop. 

In [12]:
epoch_iterator = tqdm(range(number_of_epochs))
for epoch_index in epoch_iterator:
    for x, y in zip(training_input_matrix, training_output_vector):
        y_hat = np.sum(np.multiply(y, weights) + biases)
        loss = (y_hat - y) ** 2
        sgd.take_training_step(loss)
    epoch_iterator.set_description(f'Loss: {loss.sum().data:.8f}')
    epoch_iterator.refresh()

Loss: 0.01085207: 100%|██████████| 800/800 [00:21<00:00, 37.41it/s]


Let's now see how our model performs on our test set.

In [13]:
mean_testing_loss = 0
for x, y in zip(testing_input_matrix, testing_output_vector):
    y_hat = np.sum(np.multiply(y, weights) + biases)
    loss = (y_hat - y) ** 2
    mean_testing_loss += loss.data
mean_testing_loss /= len(testing_output_vector)

In [14]:
mean_testing_loss

0.00506999587106785

Let's look at a few random results to sanity check our mode. 

In [15]:
random_test_examples = random.sample(list(zip(testing_input_matrix, testing_output_vector)), 10)
for x, y in random_test_examples:
    y_hat = np.sum(np.multiply(y, weights) + biases)
    print()
    print(f'y_hat: {y_hat.data}')
    print(f'y:     {y}')


y_hat: 42.41191004993169
y:     42.4

y_hat: 41.11768697722359
y:     41.1

y_hat: 13.83944682937596
y:     13.7

y_hat: 53.46258397843931
y:     53.5

y_hat: 14.536336176218784
y:     14.4

y_hat: 40.22168638842568
y:     40.2

y_hat: 44.004799985572426
y:     44.0

y_hat: 24.49189827397339
y:     24.4

y_hat: 34.64657161368309
y:     34.6

y_hat: 40.520353251358316
y:     40.5
